In [237]:
from scripts.path_finders import get_context_shp_path, get_data_dir, get_division_path, get_indicators_path
import pandas as pd
import shapefile
from simpledbf import Dbf5
from scripts.geo_utils import iter_shp_as_shapely, get_shapely_shape
from scripts.land_use import find_containing_radios, conn_postgis_db
import pandas as pd
from simpledbf import Dbf5

In [238]:
rus_shp_path = get_context_shp_path("rus-ciudad-2010-2011")
rus_csv_path = os.path.join(get_data_dir(), "uso-suelo-2011.csv")
clasif_path = os.path.join(get_data_dir(), "RUS_clasif_rama1.xlsx")
parcelas_csv = os.path.join(get_data_dir(), "parcelas-100112-wgs84.csv")

In [239]:
df = pd.read_csv(rus_csv_path, encoding="utf-8", sep=";")
df_clasif = pd.read_excel(clasif_path)

In [240]:
df.head()

,x,y,id,seccion,manzana,parcela,sm,smp,unificacio,parcelas_u,...,d41,d31,d21,barrio,comuna,zona,ano,rama1,sub_rama1,ss_rama1
0,-58.4936,-34.5737,562177,63,066,020B,063-066,063-066-020B,NaN,NaN,...,0,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN
1,-58.4898,-34.5709,562178,63,116,001G,063-116,063-116-001G,NaN,NaN,...,0,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN
2,-58.4892,-34.5714,562179,63,116,006B,063-116,063-116-006B,NaN,NaN,...,0,0,0,VILLA URQUIZA,12,OESTE,2011,EDIFICIOS,VIVIENDA,NaN
3,-58.4927,-34.5732,562180,63,079,022,063-079,063-079-022,NaN,NaN,...,0,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN
4,-58.4935,-34.5720,562181,63,080,029,063-080,063-080-029,NaN,NaN,...,0,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN


In [241]:
df_clasif.head()

,rama1,count,clasificacion
0,EDIFICIOS,284668,RESIDENCIAL
1,GARAGE PRIVADO,112170,RESIDENCIAL
2,COMERCIO,56608,COMERCIAL
3,LOCAL CERRADO,28805,OTROS
4,SERVICIOS COMPLEMENTARIOS AL TRANSPORTE,10310,SERVICIOS


In [242]:
serie_clasif = df_clasif.set_index("rama1")["clasificacion"]
serie_clasif.head()

rama1
EDIFICIOS                                  RESIDENCIAL
GARAGE PRIVADO                             RESIDENCIAL
COMERCIO                                     COMERCIAL
LOCAL CERRADO                                    OTROS
SERVICIOS COMPLEMENTARIOS AL TRANSPORTE      SERVICIOS
Name: clasificacion, dtype: object

In [243]:
print "Sólo", len(df[pd.isnull(df["rama1"])]), "parcelas no tienen un valor de rama1."

Sólo 7 parcelas no tienen un valor de rama1.


In [244]:
df = df.join(serie_clasif, on="rama1")
df.head()

,x,y,id,seccion,manzana,parcela,sm,smp,unificacio,parcelas_u,...,d31,d21,barrio,comuna,zona,ano,rama1,sub_rama1,ss_rama1,clasificacion
0,-58.4936,-34.5737,562177,63,066,020B,063-066,063-066-020B,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL
1,-58.4898,-34.5709,562178,63,116,001G,063-116,063-116-001G,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL
2,-58.4892,-34.5714,562179,63,116,006B,063-116,063-116-006B,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,EDIFICIOS,VIVIENDA,NaN,RESIDENCIAL
3,-58.4927,-34.5732,562180,63,079,022,063-079,063-079-022,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL
4,-58.4935,-34.5720,562181,63,080,029,063-080,063-080-029,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL


* *Calcular el AREA de cada parcela* 
    * `ALTER TABLE parcelas ADD area float;`
    * `UPDATE parcelas SET area = ST_Area(geom);`
* *Asignar el radio que contiene a la parcela* 
    * `ALTER TABLE parcelas ADD CO_FRAC_RA str;`
    * `UPDATE parcelas SET CO_FRAC_RA = r.CO_FRAC_RA FROM radios_censo_2010 r WHERE ST_Contains(r.geom, parcelas.geom);`
* *Convertir tabla PostGIS en shp:* `pgsql2shp -f parcelas -h localhost -p 5432 abenassi "SELECT * FROM parcelas;`

In [245]:
parcelas_dbf = Dbf5("shp/divisiones/parcelas_joined/parcelas.dbf")

In [246]:
parcelas_df = parcelas_dbf.to_dataframe()

In [247]:
parcelas_df.head()

,FEATID1,MANZANA,OBS,PARCELA,SECCION,SMP,WEBLINK,PARC_ESQ,ROT_SEC,ROT_PAR,SM,PARTIDA,AREA,CO_FRAC_RA
0,389,047,INFORMACION CONSISTENTE,006,001,001-047-006,001-047-006,N,01,NaN,001-047,220257,364.122743,1_9_12
1,388,047,INFORMACION CONSISTENTE,008,001,001-047-008,001-047-008,N,01,NaN,001-047,220255,280.865662,1_9_12
2,387,048,INFORMACION CONSISTENTE,008i,001,001-048-008i,001-048-008i,N,01,NaN,001-048,0,395.447053,1_9_11
3,386,048,INFORMACION CONSISTENTE,016,001,001-048-016,001-048-016,N,01,NaN,001-048,220277,640.551576,1_9_11
4,385,048,INFORMACION CONSISTENTE,031,001,001-048-031,001-048-031,N,01,NaN,001-048,220293,966.103426,1_9_11


In [248]:
print len(parcelas_df[pd.isnull(parcelas_df["CO_FRAC_RA"])]), "parcelas no están contenidas por un radio."

778 parcelas no están contenidas por un radio.


In [249]:
print df.columns[7]
cols = list(df.columns)
cols[7] = cols[7].upper()
df.columns = cols
df.columns

smp


Index([            u'x',             u'y',            u'id',       u'seccion',
             u'manzana',       u'parcela',            u'sm',           u'SMP',
          u'unificacio',    u'parcelas_u',       u'calle_1',           u'num',
             u'calle_2',        u'tipo_1',        u'tipo_2',         u'pisos',
              u'nombre',           u'd51',           u'd41',           u'd31',
                 u'd21',        u'barrio',        u'comuna',          u'zona',
                 u'ano',         u'rama1',     u'sub_rama1',      u'ss_rama1',
       u'clasificacion'],
      dtype='object')

In [250]:
df["SMP"] = df["SMP"].str.upper()
parcelas_df["SMP"] = parcelas_df["SMP"].str.upper()

In [251]:
df = df.join(parcelas_df[["SMP", "AREA", "CO_FRAC_RA"]].set_index("SMP"), on="SMP")
df.head()

,x,y,id,seccion,manzana,parcela,sm,SMP,unificacio,parcelas_u,...,barrio,comuna,zona,ano,rama1,sub_rama1,ss_rama1,clasificacion,AREA,CO_FRAC_RA
0,-58.4936,-34.5737,562177,63,066,020B,063-066,063-066-020B,NaN,NaN,...,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,251.412082,12_16_1
1,-58.4898,-34.5709,562178,63,116,001G,063-116,063-116-001G,NaN,NaN,...,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,129.608734,12_16_3
2,-58.4892,-34.5714,562179,63,116,006B,063-116,063-116-006B,NaN,NaN,...,VILLA URQUIZA,12,OESTE,2011,EDIFICIOS,VIVIENDA,NaN,RESIDENCIAL,210.973922,12_16_3
3,-58.4927,-34.5732,562180,63,079,022,063-079,063-079-022,NaN,NaN,...,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,278.004430,12_16_1
4,-58.4935,-34.5720,562181,63,080,029,063-080,063-080-029,NaN,NaN,...,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,421.064637,12_16_1


In [252]:
print len(df), "parcelas con RUS."
print len(df[pd.isnull(df["CO_FRAC_RA"])]), "parcelas sin radio asignado."
print len(df[pd.isnull(df["AREA"])]), "parcelas sin area calculada."

555755 parcelas con RUS.
11632 parcelas sin radio asignado.
10538 parcelas sin area calculada.


In [253]:
df["AREA_POR_USO"] = df["AREA"] / df.groupby(by="SMP").count()["AREA"]

In [254]:
df[df["CO_FRAC_RA"] == "12_16_1"].groupby(by="clasificacion").sum()["AREA"]

clasificacion
COMERCIAL       3812.530654
OTROS           5292.368953
RESIDENCIAL    90414.301674
SERVICIOS       5381.848152
Name: AREA, dtype: float64

In [255]:
uso_por_radio = df.groupby(by=["CO_FRAC_RA", "clasificacion"]).sum()["AREA"] / df.groupby(by="CO_FRAC_RA").sum()["AREA"]
uso_por_radio[:20]

CO_FRAC_RA  clasificacion
10_10_1     COMERCIAL        0.038155
            OTROS            0.035857
            RESIDENCIAL      0.843454
            SERVICIOS        0.082533
10_10_10    COMERCIAL        0.045252
            OTROS            0.028980
            RESIDENCIAL      0.891986
            SERVICIOS        0.033782
10_10_11    COMERCIAL        0.017029
            INDUSTRIAL       0.004169
            OTROS            0.031078
            RESIDENCIAL      0.902656
            SERVICIOS        0.045068
10_10_12    COMERCIAL        0.051756
            INDUSTRIAL       0.009026
            OTROS            0.089479
            RESIDENCIAL      0.763907
            SERVICIOS        0.085832
10_10_2     COMERCIAL        0.019215
            INDUSTRIAL       0.005497
Name: AREA, dtype: float64

In [256]:
print len(uso_por_radio.groupby(level=0)), "radios tienen el uso del suelo calculado"

3439 radios tienen el uso del suelo calculado


In [257]:
from scripts.create_indicators import get_or_create_indicators_df

indicators_radio = get_or_create_indicators_df("RADIO")
old_index = indicators_radio.index.copy()
indicators_radio = indicators_radio.set_index("CO_FRAC_RA")

In [258]:
def get_land_use(radio, use, serie):
    if not radio in serie:
        return pd.np.nan
    
    if use in serie[radio]:
        return serie[radio][use]
    else:
        return 0

In [259]:
from functools import partial

In [260]:
get_comercial_use = partial(get_land_use, use="COMERCIAL", serie=uso_por_radio)
get_industrial_use = partial(get_land_use, use="INDUSTRIAL", serie=uso_por_radio)
get_servicios_use = partial(get_land_use, use="SERVICIOS", serie=uso_por_radio)
get_otros_use = partial(get_land_use, use="OTROS", serie=uso_por_radio)

In [261]:
indicators_radio["COMERCIAL"] = map(get_comercial_use, indicators_radio.index)
indicators_radio["INDUSTRIAL"] = map(get_industrial_use, indicators_radio.index)
indicators_radio["SERVICIOS"] = map(get_servicios_use, indicators_radio.index)
indicators_radio["OTROS"] = map(get_otros_use, indicators_radio.index)

In [262]:
indicators_radio.tail()

,empleo,desocup,inact,ocup_viv,0_14,15_64,mas_65,hab,con_satisf,con_basica,...,reach_area,COMERCIAL,INDUSTRIAL,SERVICIOS,OTROS,educ_priv,educ_pub,hospitales,esp_verde,nse_alt
CO_FRAC_RA,,,,,,,,,,,,,,,,,,,,,
15_21_9,0.740891,0.035149,0.238866,0.746063,0.141367,0.698725,0.159907,863,0.796834,0.203166,...,42.880817,0.157499,0.000000,0.078906,0.015775,0,0,0,0,3
15_21_10,0.716763,0.031250,0.267823,0.708010,0.153344,0.698206,0.148450,613,0.868613,0.062044,...,41.830951,0.073954,0.000000,0.093680,0.012688,0,0,0,0,3
15_21_11,0.811037,0.030000,0.168896,0.739726,0.142037,0.736011,0.121951,697,0.956790,0.030864,...,16.986219,0.056333,0.009658,0.102227,0.029582,0,0,0,0,3
15_21_12,0.760820,0.037464,0.216401,0.778986,0.139216,0.735294,0.125490,510,0.860465,0.074419,...,16.858104,0.029522,0.000000,0.209995,0.011504,0,0,0,0,3
15_21_13,0.691517,0.046099,0.244216,0.770161,0.130726,0.689385,0.179888,895,0.869110,0.099476,...,50.842914,0.132925,0.017719,0.198981,0.114428,0,0,0,0,3


In [263]:
radio_index = indicators_radio.index.copy()
indicators_radio = indicators_radio.set_index(old_index)
indicators_radio["CO_FRAC_RA"] = radio_index
indicators_radio.to_csv(get_indicators_path("RADIO"), encoding="utf-8")